# 🪙💲 G-Research - Full overlap-exploiting ("leaky") solution
![](https://storage.googleapis.com/kaggle-competitions/kaggle/30894/logos/header.png)


&nbsp;
&nbsp;
&nbsp;
&nbsp;

---
# 👇👇👇
# [🪙💲 Proposal for a meaningful LB + LGBM [S]](https://www.kaggle.com/julian3833/s-proposal-for-submission-common-ground)
# 👆👆👆 
# A proposal for a method to get comparable, non-leaky models 
# + one non-leaky model scoring `LB=0.018`

---

&nbsp;
&nbsp;
&nbsp;
&nbsp;


### After the early dissapointment with [my original model](https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline-lb)'s LB contamination and while I make some time to work on a more reasonable starter model, I thought: The LB is _already_ contaminated and there is no going back, so why not? It is just a matter or time, and, actually, the LB, in the current situation, is meaningless.

### Below you can see a 18-line submission that overfits the public LB fully, using the fact that the test period is contained in the train data to get the closest point for each test row.

It is not _actually_ leaky because the fact that the LB is part of the public data was publicly disclosed by the organizers. In my opinion, though, the statement was not clear enough regarding the fact that the public LB test period was contained in the `train.csv`, but I guess it doesn't matter.

I don't know the public test time range, but here I'm assuming that is starts from the placeholder and goes on.
The placeholder `test.csv` starts at `'2021-06-13 00:00:00'`. I tried with the full `train.csv` but it takes a long time and it's probably going to fail the submission.

The LB doesn't seem to be meaningful at all as long as we don't know which training data points are leaking into the public test set and which data points are not. See [this discussion](https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/285289)

References: [Stackoverflow: Pandas DataFrame How to query the closest datetime index?](https://stackoverflow.com/questions/42264848/pandas-dataframe-how-to-query-the-closest-datetime-index).



----

# See also:
* __[Watch out!: test LB period is contained in the train csv](https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/285505) (topic)__


Be aware of this situation if you forked one of the early versions of [🪙💲 G-Research - Starter LGBM Pipeline](https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline-lb) as well.



In [ ]:
import numpy as np
import pandas as pd
import gresearch_crypto
env = gresearch_crypto.make_env()
iter_test = env.iter_test()
df = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv', 
                 usecols=['Target', 'Asset_ID','timestamp'], dtype={'Asset_ID': 'int8'})
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
df = df.set_index('datetime').drop('timestamp', axis=1)
df = df[(df.index.year == 2021) & (df.index.month > 5)]
dfs = {asset_id: df[df['Asset_ID'] == asset_id].resample('1min').interpolate().copy() for asset_id in df['Asset_ID'].unique()}
del df
for df_test, df_pred in iter_test:
    df_test['datetime'] = pd.to_datetime(df_test['timestamp'], unit='s')
    for _, row in df_test.iterrows():
        try:
            df = dfs[row['Asset_ID']]
            closest_train_sample = df.iloc[df.index.get_loc(row['datetime'], method='nearest')]
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = closest_train_sample['Target']
        except:
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
    df_pred['Target'] = df_pred['Target'].fillna(0)
    env.predict(df_pred)